In [1]:
import pandas as pd
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import LabelEncoder

from sklearn.covariance import EllipticEnvelope

In [2]:
df_devices = pd.read_csv("all_devices.csv")
df_devices["id"] = df_devices["network_id"].map(str)+"_"+df_devices["device_id"].map(str)
print(df_devices.shape)
print(df_devices.nunique())
df_devices.to_csv("all_devices_v1.csv.gz",compression="gzip",index=False)
df_devices.head()

(76261, 9)
Unnamed: 0                  76261
network_id                      4
device_id                   71663
type                            7
model                        1357
manufacturer                  263
operating_system               80
operating_system_version      266
id                          76261
dtype: int64


,Unnamed: 0,network_id,device_id,type,model,manufacturer,operating_system,operating_system_version,id
0,113,0,1028623,MOBILE_PHONE,Galaxy S8,Samsung,Android,9,0_1028623
1,587,0,48047,MOBILE_PHONE,Galaxy Note 8,Samsung,Android,9,0_48047
2,668,0,123568,MOBILE_PHONE,H918,LG Electronics,Android,8.0.0,0_123568
3,830,0,95366,MOBILE_PHONE,iPhone 6,"Apple, Inc.",iOS,NaN,0_95366
4,886,0,1755023,TABLET,iPad,Apple,iOS,NaN,0_1755023


In [3]:
df_devices.isna().sum(axis=1)

0        0
1        0
2        0
3        1
4        1
        ..
76256    0
76257    0
76258    3
76259    0
76260    0
Length: 76261, dtype: int64

In [4]:
%%time
df = pd.read_csv("all_sessions.csv")
df.drop("Unnamed: 0",axis=1,inplace=True)
df["id"] = df["network_id"].map(str)+"_"+df["device_id"].map(str)
df["row_nan_count"] = df.isna().sum(axis=1)
print(df.shape)
# print(df_devices.nunique())
df.tail()

(22517480, 39)
CPU times: user 1min 54s, sys: 44.6 s, total: 2min 38s
Wall time: 2min 45s


,network_id,device_id,timestamp,host,host_ip,port_dst,transport_protocol,service_device_id,packets_count,outbound_bytes_count,inbound_bytes_count,packet_loss,retransmit_count,latency,session_count,outbound_packets_count,inbound_packets_count,outbound_bytes_max,outbound_bytes_min,outbound_bytes_mean,outbound_bytes_median,outbound_bytes_stddev,inbound_bytes_max,inbound_bytes_min,inbound_bytes_mean,inbound_bytes_median,inbound_bytes_stddev,outbound_packet_size_max,outbound_packet_size_min,outbound_packet_size_mean,outbound_packet_size_median,outbound_packet_size_stddev,inbound_packet_size_max,inbound_packet_size_min,inbound_packet_size_mean,inbound_packet_size_median,inbound_packet_size_stddev,id,row_nan_count
22517475,3,2701197,1565287200,3711f5efef9f8d50bc4abdd75c7cbe87c9f838706a147cbfa1c26995fc1d2a7c,a94ecd6efd1fc34c84e046fe5d6dee755db1eaac9e1f5955fcf0aa62646f9eec,443,TCP,NaN,19,2311,0,0.00000,1,NaN,1,19,0,2311,2311,2311.00000,NaN,NaN,0,0,0.00000,NaN,NaN,121.63158,121.63158,121.63158,121.63158,NaN,NaN,NaN,NaN,NaN,NaN,3_2701197,12
22517476,3,2701266,1565283600,c1a3faaa82c5e49715eb0fee31a95f06b5bd561df79b61f98f217530cece302e,e64f49ebacd18d4873e72c53a19ecf01f44d162b3ba46f1413e8006d69464b9b,443,TCP,700571.0,58,1852,3906,0.00000,0,NaN,7,29,29,451,139,264.57143,NaN,156.99879,1353,135,558.00000,NaN,565.68395,69.50000,56.37500,66.41752,69.50000,5.27282,216.33333,67.5,108.22874,67.5,60.64558,3_2701266,3
22517477,3,2702227,1565287200,dab96d3e8508c44a1787d8ac1c8ea45ac5e6a96b4dc5aaa5c778532ebc246d71,dab96d3e8508c44a1787d8ac1c8ea45ac5e6a96b4dc5aaa5c778532ebc246d71,1146,TCP,2305795.0,114,2337,2964,0.00000,0,NaN,57,57,57,41,41,41.00000,NaN,0.00000,52,52,52.00000,NaN,0.00000,41.00000,41.00000,41.00000,41.00000,0.00000,52.00000,52.0,52.00000,52.0,0.00000,3_2702227,3
22517478,3,2703918,1565298000,d8495e90103789563d9dd8d4700f4304702454ee0bc5a865e9cf722c30b44337,d8495e90103789563d9dd8d4700f4304702454ee0bc5a865e9cf722c30b44337,53,UDP,NaN,14,535,849,NaN,0,NaN,7,7,7,103,58,76.42857,NaN,16.17464,168,80,121.28571,NaN,35.52330,103.00000,58.00000,76.42857,72.00000,16.17464,168.00000,80.0,121.28571,131.0,35.52330,3_2703918,5
22517479,3,2710107,1565287200,11753523a4b797700bd58cfff461e53759278bf9dc588c0e8fec888471f0ca61,9346cc4b9bebc133df1c65b39dacf6efd092575377e2cb6f8df461ba71ca35b8,443,TCP,NaN,22,4039,0,0.16458,1,NaN,1,22,0,4039,4039,4039.00000,NaN,NaN,0,0,0.00000,NaN,NaN,183.59091,183.59091,183.59091,183.59091,NaN,NaN,NaN,NaN,NaN,NaN,3_2710107,12


In [5]:
print(df.host.nunique())
enc = LabelEncoder()
df.host = enc.fit_transform(df.host)
print(df.host.nunique())
df.head()

447308
447308


,network_id,device_id,timestamp,host,host_ip,port_dst,transport_protocol,service_device_id,packets_count,outbound_bytes_count,inbound_bytes_count,packet_loss,retransmit_count,latency,session_count,outbound_packets_count,inbound_packets_count,outbound_bytes_max,outbound_bytes_min,outbound_bytes_mean,outbound_bytes_median,outbound_bytes_stddev,inbound_bytes_max,inbound_bytes_min,inbound_bytes_mean,inbound_bytes_median,inbound_bytes_stddev,outbound_packet_size_max,outbound_packet_size_min,outbound_packet_size_mean,outbound_packet_size_median,outbound_packet_size_stddev,inbound_packet_size_max,inbound_packet_size_min,inbound_packet_size_mean,inbound_packet_size_median,inbound_packet_size_stddev,id,row_nan_count
0,0,35,1565074800,413484,ecbb92cd941972b779d18451b6f96275587941e4cf07a1715b708f87e917fdf1,49152,TCP,790889.0,260,21855,7815,0.0675,67,NaN,39,235,25,930,465,560.38462,NaN,190.21935,1563,0,200.38462,NaN,529.37001,93.00000,93.00000,93.00000,93.00000,0.00000,312.6,312.6,312.6,312.6,0.0,0_35,3
1,0,35,1565053200,252449,90cf529b11c8f26efbb3936c7d10a5bf57c1a930603af00f09c9f4e08b5036d5,49153,TCP,1604622.0,178,8326,28656,0.0000,16,NaN,18,88,90,467,427,462.55556,467.0,12.93523,1592,1592,1592.00000,1592.0,0.00000,106.75000,93.40000,94.88333,93.00000,4.31713,318.4,318.4,318.4,318.0,0.0,0_35,1
2,0,57,1565082000,393482,d43dad76e6cef2231d2efc743e498996b40f8b13fc120bb96b06412231602efa,443,TCP,NaN,67,4013,0,0.0000,40,NaN,7,67,0,3521,82,573.28571,NaN,1299.81982,0,0,0.00000,NaN,0.00000,64.01818,41.00000,44.28831,41.00000,8.70005,NaN,NaN,NaN,NaN,NaN,0_57,9
3,0,57,1565082000,46068,945e37dab8aee93dd4e650f8d17d76a3adfbc6aa70ebba379e65700bc61efa17,443,TCP,NaN,45,9158,0,0.0000,3,NaN,4,45,0,3401,935,2289.50000,2411.0,1018.07449,0,0,0.00000,0.0,0.00000,226.73333,133.57143,194.96824,209.00000,41.77727,NaN,NaN,NaN,NaN,NaN,0_57,7
4,0,57,1565082000,389390,4b43e85e630c2d18a0afaa2a6366367c4fc52d32b4ba5b54517b808e2581d88c,443,TCP,NaN,9,1671,0,0.0000,0,NaN,1,9,0,1671,1671,1671.00000,NaN,NaN,0,0,0.00000,NaN,NaN,185.66667,185.66667,185.66667,185.66667,NaN,NaN,NaN,NaN,NaN,NaN,0_57,12


In [6]:
print(df.host_ip.nunique())
enc = LabelEncoder()
df.host_ip = enc.fit_transform(df.host_ip)
print(df.host_ip.nunique())
df.head()

386180
386180


,network_id,device_id,timestamp,host,host_ip,port_dst,transport_protocol,service_device_id,packets_count,outbound_bytes_count,inbound_bytes_count,packet_loss,retransmit_count,latency,session_count,outbound_packets_count,inbound_packets_count,outbound_bytes_max,outbound_bytes_min,outbound_bytes_mean,outbound_bytes_median,outbound_bytes_stddev,inbound_bytes_max,inbound_bytes_min,inbound_bytes_mean,inbound_bytes_median,inbound_bytes_stddev,outbound_packet_size_max,outbound_packet_size_min,outbound_packet_size_mean,outbound_packet_size_median,outbound_packet_size_stddev,inbound_packet_size_max,inbound_packet_size_min,inbound_packet_size_mean,inbound_packet_size_median,inbound_packet_size_stddev,id,row_nan_count
0,0,35,1565074800,413484,357105,49152,TCP,790889.0,260,21855,7815,0.0675,67,NaN,39,235,25,930,465,560.38462,NaN,190.21935,1563,0,200.38462,NaN,529.37001,93.00000,93.00000,93.00000,93.00000,0.00000,312.6,312.6,312.6,312.6,0.0,0_35,3
1,0,35,1565053200,252449,218408,49153,TCP,1604622.0,178,8326,28656,0.0000,16,NaN,18,88,90,467,427,462.55556,467.0,12.93523,1592,1592,1592.00000,1592.0,0.00000,106.75000,93.40000,94.88333,93.00000,4.31713,318.4,318.4,318.4,318.0,0.0,0_35,1
2,0,57,1565082000,393482,319956,443,TCP,NaN,67,4013,0,0.0000,40,NaN,7,67,0,3521,82,573.28571,NaN,1299.81982,0,0,0.00000,NaN,0.00000,64.01818,41.00000,44.28831,41.00000,8.70005,NaN,NaN,NaN,NaN,NaN,0_57,9
3,0,57,1565082000,46068,223744,443,TCP,NaN,45,9158,0,0.0000,3,NaN,4,45,0,3401,935,2289.50000,2411.0,1018.07449,0,0,0.00000,0.0,0.00000,226.73333,133.57143,194.96824,209.00000,41.77727,NaN,NaN,NaN,NaN,NaN,0_57,7
4,0,57,1565082000,389390,113695,443,TCP,NaN,9,1671,0,0.0000,0,NaN,1,9,0,1671,1671,1671.00000,NaN,NaN,0,0,0.00000,NaN,NaN,185.66667,185.66667,185.66667,185.66667,NaN,NaN,NaN,NaN,NaN,NaN,0_57,12


In [7]:
df.shape

(22517480, 39)

In [8]:
# df.drop_duplicates().shape ## 22463971

In [9]:
df.drop_duplicates(inplace=True)

KeyboardInterrupt: 

In [ ]:
df["service_device_id_count"] = df.groupby("service_device_id")["packets_count"].transform("size")
df["host_count"] = df.groupby("host")["packets_count"].transform("size")
df["device_id_count"] = df.groupby("device_id")["packets_count"].transform("size")
df["id-service_device_counts"] = df.groupby(["id","device_id"])["packets_count"].transform("size")
df["port_dst_count"] = df.groupby("port_dst")["packets_count"].transform("size")

In [ ]:
df.to_csv("all_sessions_v1.csv.gz",compression="gzip",index=False)

In [ ]:
df.sample(223456).to_csv("400k_sample_sessions_v1.csv.gz",compression="gzip",index=False)

# Baseline

# basic features + sub
* base on baseline

* features per network
* orig: As we mentioned before we are going to select only 5 features: unique hosts count, unique port count, total packets transferred, total inbound bytes and total outbound bytes. Lets create the feature set for each network

In [ ]:
list(df.columns)

In [ ]:
df.head()

In [ ]:
df.latency.describe()

In [18]:
def extract_features(df):
    return df.groupby(["device_id"]).aggregate({"host": "nunique",
                                                "port_dst": "nunique",
                                                 'row_nan_count':"max",
                                                 'row_nan_count':"min",
                                                "service_device_id": "nunique",
 'service_device_id_count':"max",
 'service_device_id_count':"min",
 'host_count':["min","max"],
 'device_id_count':["min","max"],
 'id-service_device_counts':["min","max"],
 'port_dst_count':"max",'port_dst_count':"min",
                                                
                                                "host_ip": "nunique",
                                                "packets_count":["sum","mean"], "inbound_bytes_count": "sum", 
                                                "outbound_bytes_count": "sum",
                                               "latency":["max","min","std"]})

In [19]:
def extract_features(df):
    return df.groupby(["device_id"]).aggregate({"host": "nunique",
                                                "port_dst": "nunique",
                                                 'row_nan_count':"max",
                                                 'row_nan_count':"min",
                                                "service_device_id": "nunique",
 'host_count':["min","max"],
 'device_id_count':["min","max"],
 'port_dst_count':"max",'port_dst_count':"min",
                                                
                                                "host_ip": "nunique",
                                                "packets_count":["sum","mean"], "inbound_bytes_count": "sum", 
                                                "outbound_bytes_count": "sum",
                                               "latency":["max","min","std"]})

In [20]:
networks_fs = []

for i in range(4):
    print(i)
    networks_fs.append(extract_features(df[df.network_id == i]))

0


KeyError: "Column 'host_count' does not exist!"

In [ ]:
### fill nans
for i in range(4):
#     networks_fs[i].fillna(-1,inplace=True)
    networks_fs[i].dropna(axis=1,inplace=True)

### Modeling Fun
* As we've mentioned we used the Elliptic Envelope model for our anomaly detection. We use the decision_function in order to receive the confidence and we normalize the values to values between 0 and 1.

In [10]:
# We use the simple min-max normalization in order to normalize the confidence values to 0-1 range.
# Higher score means that this device is probably more anomalous.
def calc_normalized_decision(decision_function_result):
    decision_function_result = -1 * decision_function_result
    minimum = decision_function_result.min()
    maximum = decision_function_result.max()
    return (decision_function_result - minimum) / (maximum - minimum)

def detect_anomalies(feature_set, param = 0.05):
    ee = EllipticEnvelope(contamination=param).fit(feature_set.values)
    decision_function_result = ee.decision_function(feature_set.values)
    feature_set["confidence"] = calc_normalized_decision(decision_function_result)
    return feature_set

In [17]:
def by_force(param):
    networks_fs = []

    for i in range(4):
        print(i)
        networks_fs.append(extract_features(df[df.network_id == i]))
    ### fill nans
    for i in range(4):
    #     networks_fs[i].fillna(-1,inplace=True)
        networks_fs[i].dropna(axis=1,inplace=True)
    for i in range(4):
        network_fs[i] = detect_anomalies(networks_fs[i], param)

    # Remove the device_id from index and Add network id to each data set
    for i in range(4):
        networks_fs[i].reset_index(level=0, inplace=True)
        networks_fs[i]["network_id"] = i ## bug??    
    df_to_submit = pd.concat(networks_fs)[["network_id", "device_id", "confidence"]] # The column order is important!
    df_to_submit.head()
    arr_to_submit = df_to_submit.to_json(orient='values')
    from urllib import request
    import json

    leaderboard_name = "armis"
    host = "leaderboard.datahack.org.il"

    # Name of the user
    submitter = "Overfitterss"

    predictions = json.loads(arr_to_submit)

    jsonStr = json.dumps({'submitter': submitter, 'predictions': predictions})
    data = jsonStr.encode('utf-8')
    req = request.Request(f"https://{host}/{leaderboard_name}/api/",
                          headers={'Content-Type': 'application/json'},
                          data=data)
    resp = request.urlopen(req)
    
    ret = json.load(resp)
    return ret['score']
max_score = 0
max_param = 0.05
for i in [0.05, 0.06]:    
    score = by_force(i)
    if score > max_score:
        max_score = score
        max_param = i 
    else:   
        print('resubmitting')
        score = by_force(i)        

0


KeyError: "Column 'host_count' does not exist!"

In [ ]:
# Remove the device_id from index and Add network id to each data set
for i in range(4):
    networks_fs[i].reset_index(level=0, inplace=True)
    networks_fs[i]["network_id"] = i ## bug??

In [ ]:
df_to_submit = pd.concat(networks_fs)[["network_id", "device_id", "confidence"]] # The column order is important!
df_to_submit.head()

## Submissions
* In order to update the Leader Board you need to send a POST request to following url: "https://leaderboard.datahack.org.il/armis/api/". Our Leader Board receives your anomalies results in the json form of [["network_id", "device_id", "confidence"]] - The order is important!

In [ ]:
arr_to_submit = df_to_submit.to_json(orient='values')

In [ ]:
from urllib import request
import json

leaderboard_name = "armis"
host = "leaderboard.datahack.org.il"

# Name of the user
submitter = "Overfitterss"

predictions = json.loads(arr_to_submit)

jsonStr = json.dumps({'submitter': submitter, 'predictions': predictions})
data = jsonStr.encode('utf-8')
req = request.Request(f"https://{host}/{leaderboard_name}/api/",
                      headers={'Content-Type': 'application/json'},
                      data=data)
resp = request.urlopen(req)
print(json.load(resp))